## ESM - another general algebraic modelling system
Python package for generating and handling algebraic convex linear programs 


- - -
PACKAGE LOCAL INSTALLATION and USAGE

Editable local installation:
1. Create a virtual environment based on 'environment.yml' file.
2. In the cmd, from the package root, run: >>> python setup.py sdist bdist_wheel
2. In the cmd, from the package root, run: >>> python -m pip install -e .
3. From the virtual environment: >>> import esm
4. Use esm APIs (Model class, create_model_dir method)


- - -
CREATE/UPDATE ENVIRONMENT BASED ON YML FILE IN PROJECT REPO

In the prompt:
- From the path where environment.yml is present: >>> conda env create -f environment.yml
- The environment is named "esm", so type >>> conda activate esm

UPDATE ENVIRONMENT YML FILE (in case of modifications)

In the prompt: 
- activate the working environment: >>> conda activate your_environment_name
- export environment.yml file based on the working environment: >>> conda env export > environment.yml


- - - 
IMPORT PACKAGE, DEFINE MODEL NAME AND ROOT PATH

In [ ]:
import esm

model_dir_name = 'pyesm_model'
main_dir_path = r'D:\Politecnico di Milano\DENG-PRIN-MIMO - Documenti\Models\model_physical\pyesm_model'

- - - 
MODEL DIRECTORY GENERATION

Generation of a model directory in a defined path. Notes:
- Blank .yml or .xlsx files can be generated
- API_usage_guide.ipynb can be generated as guidance for main APIs
- Template models can be imported based on the tests model gallery [tbd]



In [ ]:
esm.create_model_dir(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    template_file_type='xlsx',
    export_tutorial=True,
)

- - - 
MODEL GENERATION FROM SCRATCH

Step-by-step model creation with sets, data and problems generation.

- Generation of a Model instance defined by settings files filled by user
- model_settings_from: select among yml and xlsx
- use_existing_data=True : model coordinates are loaded and numerical problem initialized (model ready to be run). 
- use_existing_data=False : sets excel file generated only, to be filled by user

In [ ]:
model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    log_level='debug',
    model_settings_from='xlsx',
    multiple_input_files=True,
    use_existing_data=False,
    detailed_validation=True,
)

Once sets.xlsx file has filled with sets data:

- load_model_coordinates(): load user defined sets, fill coordinates
- initialize_blank_data_structure():
    - generate blank sqlite database with sets and variables (empty numerical values)
    - generate blank xlsx files for exogenous data to be filled by the user

In [ ]:
model.load_model_coordinates()
model.initialize_blank_data_structure()

Once exogenous xlsx files has filled:

- load_exogenous_data_to_sqlite_database(): load input data into sqlite database
- initialize_problems(): load and validate symbolic problem, generate numerical problem 

In [ ]:
model.load_exogenous_data_to_sqlite_database()
model.initialize_problems()

- - - 
MODEL GENERATION FROM EXISTING DATA

If use_existing_data=True, generation of Model instance working with existing database and data input files.

The model is generated and ready to be solved.

In [3]:
model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    log_level='debug',
    model_settings_from='xlsx',
    use_existing_data=True,
    detailed_validation=True,
)

INFO | Model | Generating '1_coupled_model' model instance.
DEBUG | Model.core.index | Loading and validating 'structure_sets' data structure from 'xlsx' source.
DEBUG | Model.file_manager | Excel tab 'structure_sets' loaded from 'model_settings.xlsx'.
DEBUG | Model.core.index | Loading and validating 'structure_variables' data structure from 'xlsx' source.
DEBUG | Model.file_manager | Excel tab 'structure_variables' loaded from 'model_settings.xlsx'.
DEBUG | Model.core.index | Completing data tables with information taken from related Sets.
DEBUG | Model.core.index | Fetching and validating variables from data tables, generating 'Variable' objects.
DEBUG | Model.core.index | Fetching 'coordinates_info' to Index.variables.
INFO | Model | Loading existing sets data and variable coordinates to Index.
DEBUG | Model.core.index | Loading Sets data to Index.sets.
DEBUG | Model.file_manager | Excel file 'sets.xlsx' loaded.
DEBUG | Model.core.index | Loading variable coordinates to Index.data.

- - - 
SAVING/LOADING MODEL INSTANCE

In case model instance generation is taking huge time, it is possible to save/load model instance to avoid regenerating it several times.

Notice that - for unknown reasons - model instance can be loaded only if it has not already solve (i.e. do not save model instance if you run the model!)

In [ ]:
esm.save_model_instance(
    instance=model, # name of the model instance here
    file_name='instance_name'
)

In [ ]:
model = esm.load_model_instance(
    file_name='file_name',
    source_dir_path=r"D:\git_repos\pyesm\tests\models\integrated\1_coupled_model\instances",
)

- - -
DATA and SYMBOLIC PROBLEM UPDATE without regenerating Model instance

- initialize_problems(): if symbolic problem has modified, upload it and generates a new numerical model

- update_database_and_problem(): in case of modifications in input data files (but not in sets, nor in variables structures) and symbolic problem, update database and problem

- reinitialize_sqlite_database(): reinitialize sqlite database endogenous variables


In [ ]:
# in case of need for generating only one or more data input files/tabs
model.generate_input_data_files(table_key_list=[])

In [ ]:
# in case of modifications in symbolic problem only,
# update problems dataframe and symbolic problem
model.initialize_problems()

In [ ]:
# in case of modifications in input data files (but not in sets, nor in 
# variables structures) and symbolic problem, update database and problem
model.update_database_and_problem()

In [ ]:
# in case of modifications in input data files (but not in sets, nor in 
# variables structures) and symbolic problem, reinitialize sqlite database for
# endogenous variables
model.reinitialize_sqlite_database(force_overwrite=True)

- - -
SOLVE NUMERICAL PROBLEM and ENDOGENOUS DATA EXPORT

- run_model(): run model with various settings
- load_results_to_database(): once model has successfully solved, load endogenous parameters data to sqlite database.
- check_model_results(): check model results compared to an existing database (path and name of the database with the expected results in model settings attributes)

In [11]:
model.run_model(
    verbose=False,
    solver='GUROBI',
    integrated_problems=True,
)

INFO | Model | Solving '2' integrated numerical problem(s) for '3' scenarios with 'GUROBI' solver.
INFO | Model.core | =================================
INFO | Model.core | Solving integrated problems for scenario ['low energy']
DEBUG | Model.file_manager | File 'database.db' successfully copied as 'database_previous_iter.db'.
INFO | Model.core.problem | Solving cvxpy sub-problem '1' - Scenario ['low energy'].


Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-12


INFO | Model.core.problem | Solving cvxpy sub-problem '2' - Scenario ['low energy'].
DEBUG | Model.core | Exporting data from cvxpy endogenous variable (in data table) to SQLite database 'database.db' 
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
DEBUG | Model.core | Exporting data from cvxpy variable to the related data table 'x'. 
DEBUG | Model.core.sql_manager | SQLite table 'x' - 2 entries updated.
DEBUG | Model.core | Exporting data from cvxpy variable to the related data table 'a'. 
DEBUG | Model.core.sql_manager | SQLite table 'a' - 2 entries updated.
DEBUG | Model.core.sql_manager | Connection to 'database.db' closed.
DEBUG | Model.file_manager | File 'database_previous_iter.db' have been erased.
DEBUG | Model.core | Fetching data from 'database.db' to cvxpy exogenous variables.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
DEBUG | Model.core | Fetching data from table 'x' to cvxpy exogenous variable.
DEBUG | Model.core | Fetching 

In [ ]:
model.load_results_to_database()

In [ ]:
model.check_model_results()

- - -
UTILITIES

- sets(): list of model sets
- variables(): dictionary with variable name (key) and shape of teh variable (value)
- variable(name, ...): visual check of variables content in the database
- set(name): visual check of sets content in the database

In [ ]:
model.variables
model.sets

In [ ]:
model.set('products')

In [ ]:
model.variable(name='L',)